# The Battle of Neighborhood (Week 1 - 2)
### Business Problem

**Background** \
According to the annual Demographia international housing affordability survey, Sydney ranked third and Melbourne fourth compared to 309 housing markets across eight countries including some of the most expensive cities in the world such as Hong Kong or Vancouver. Melbourne's housing market is heating up, as spring is coming. “Spring is a traditional strong sales period, and with fresh demand from buyers, now is the time for anyone thinking about selling a house or a unit to act”, Real Estate Institute of Victoria CEO Gil King says. However, in such a scenario, this situation imposes a big concern for homebuyers: *Among hundred thousands available real estate on sale, which one is both most suitable and affordable for my preferences and financial status?* 

**Business Problem** 

This analysis is for anyone who is planning to buy/rent a new house/apartment in Melbourne. In my opinion, it is necesarry to adopt machine learning and data analysis to provide recommendations for them to decide or purchase a suitable and reasonably priced real estate in Melbourne. \
To solve this business problem, we are going to cluster Melbourne neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We will recommend profitable venues based on amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

Imagining there is a client coming to your door. They have 2 kids, so the house needs at least 3 rooms
They prefer an unit or duplex house, and as they are Vietnamese migrants, they want some local restaurants nearby.
Other essential and recreational amenities, especially for sports, are also welcomed. 
Capability: AUD650,000 – AUD800,000. 

As a real estate agent, how can you help them?

### Data Collection

Data on Melbourne properties and price was scraped from publicly available results posted every week from DomainGroup < https://www.domain.com.au/>. The dataset includes almost every information you request of a property, such as Address, Type of Real estate, Suburb, Method of Selling, Rooms, Price, Real Estate Agent, Date of Sale. The dataset even contains location coordinates of properties for the sake of clustering and visualizing. In order to better understand the dataset, here are some notes to look up: 

|    **Key**              |    **Meaning**                                                                 
|---------------------|-----------------------------------------------------------------------------|
|    Suburb           |    Suburb                                                                   |   
|    Rooms            |    Number of rooms                                                          |   
|    Price            |    Price in Australian   dollars                                            |   
|    Br               |    Bedroom(s)                                                               |   
|    H                |    House, cottage,   villa, semi, terrace                                   |   
|    U                |    Unit, duplex                                                             |   
|    T                |    Townhouse                                                                |   
|    Dev site         |    Development site                                                         |   
|    O res            |    Other residential                                                        |   
|    Distance         |    Distance from   Central Business District (City Centre) in kilometres    |  
|    Regionname       |    General Region (West,   North West, North, etc.)                         |  
|    Propertycount    |    Number of properties   that exist in the suburb                          |   
|    CouncilArea      |    Governing council   for the area                                         |  

To explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we will access data through FourSquare API interface and arrange them as a dataframe for visualization. By merging data on Melbourne properties and price and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we will be able to recommend suitable real estate investments.


### Methodology
The Methodology section will describe the main components of our analysis and predication system. The Methodology section starts with 2 initial stages: 
1. Data Exploration and Understanding  
2. Data Preparation and Preprocessing 

Then it is divided into two main sections:

**Section 1**: Clustering house prices into 3 groups: Low, Medium, High

**Section 2**: Get nearby venues for references

### 1. Data Exploration and Understanding

In [2]:
# The code was removed by Watson Studio for sharing.

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


In [3]:
df.shape

(34857, 21)

Our dataset consists of 34857 rows and 21 columns. This is an enormous size so we need to prepare the data to select only relevant information.

### 2. Data Prepration and Preprocessing

In [4]:
#Select only necessary information
df = df[['Suburb','Rooms','Type','Price','Bedroom2','Lattitude','Longtitude']]

#Ignore all properties without price and bedroom information 
df = df[pd.notnull(df['Price'])]
df = df[pd.notnull(df['Bedroom2'])]
df = df[pd.notnull(df['Lattitude'])]
df.head(10)

,Suburb,Rooms,Type,Price,Bedroom2,Lattitude,Longtitude
1,Abbotsford,2,h,1480000.0,2.0,-37.7996,144.9984
2,Abbotsford,2,h,1035000.0,2.0,-37.8079,144.9934
4,Abbotsford,3,h,1465000.0,3.0,-37.8093,144.9944
5,Abbotsford,3,h,850000.0,3.0,-37.7969,144.9969
6,Abbotsford,4,h,1600000.0,3.0,-37.8072,144.9941
10,Abbotsford,2,h,941000.0,2.0,-37.8041,144.9953
11,Abbotsford,3,h,1876000.0,4.0,-37.8024,144.9993
14,Abbotsford,2,h,1636000.0,2.0,-37.8060,144.9954
17,Abbotsford,1,u,300000.0,1.0,-37.8008,144.9973
18,Abbotsford,2,h,1097000.0,3.0,-37.8010,144.9989


In [5]:
#Select only properties that is "house" type with at least 2 bedrooms
df = df[(df['Rooms']>=3)&(df['Type']=='u')]
df.head(10)

,Suburb,Rooms,Type,Price,Bedroom2,Lattitude,Longtitude
41,Abbotsford,3,u,1090000.0,3.0,-37.8078,144.9965
90,Airport West,3,u,752000.0,3.0,-37.7302,144.8855
236,Altona,3,u,630000.0,2.0,-37.8702,144.8143
247,Altona,3,u,605000.0,3.0,-37.8646,144.8270
255,Altona,3,u,730000.0,3.0,-37.8680,144.8286
260,Altona,3,u,730000.0,3.0,-37.8680,144.8287
269,Altona,3,u,577000.0,3.0,-37.8684,144.8150
272,Altona,3,u,605000.0,3.0,-37.8659,144.8220
278,Altona,3,u,730000.0,3.0,-37.8680,144.8285
367,Armadale,3,u,995000.0,3.0,-37.8552,145.0160


In [6]:
df.shape

(576, 7)

After setting some conditions based on the client's preferences, we got the dataset of 18391 houses that seems suitable for her family. Now we move to next part, clustering those properties into 3 groups of price: Low, Middle, High for better recommendations.

## Section 1: Clustering into 3 groups: Low, Middle, High Price

In [41]:
#We will use the k-Means algorithm 
from sklearn.cluster import KMeans
#We consider price aspect and see if we can get a pattern
cluster_price=df[['Price']]
num_cluster = 3

#Build the k-Means model
model = KMeans(n_clusters = num_cluster)
model.fit(cluster_price)

#Predict and get cluster label
num_cluster
prediction = model.predict(cluster_price)

#Add predictions to the features df
df['Cluster'] = prediction
df.head()

,Suburb,Rooms,Type,Price,Bedroom2,Lattitude,Longtitude,Cluster
41,Abbotsford,3,u,1090000.0,3.0,-37.8078,144.9965,1
90,Airport West,3,u,752000.0,3.0,-37.7302,144.8855,0
236,Altona,3,u,630000.0,2.0,-37.8702,144.8143,0
247,Altona,3,u,605000.0,3.0,-37.8646,144.8270,0
255,Altona,3,u,730000.0,3.0,-37.8680,144.8286,0


In [8]:
df.groupby("Cluster").Price.agg(['min', 'max'])

,min,max
Cluster,,
0,301000.0,782000.0
1,1320000.0,3610000.0
2,786000.0,1296000.0


In [9]:
df.loc[df['Cluster'] == 0, df.columns[[1] + list(range(5, df.shape[1]))]].shape

(286, 4)

In [10]:
df.loc[df['Cluster'] == 1, df.columns[[1] + list(range(5, df.shape[1]))]].shape

(60, 4)

In [11]:
df.loc[df['Cluster'] == 2, df.columns[[1] + list(range(5, df.shape[1]))]].shape

(230, 4)

As we can see, the k-Means algorithm has performed unsupervised learning to divide price house into 3 clusters. Check out the table for more information:

| Cluster 	| Range (AUD) 	| Category 	| Count 	|
|:-------:	|:------------------------:	|:------------:	|:-----:	|
    | 0 	| 780,000 - 1,296,000                       	| Medium Price 	| 238 	|
    | 1 	| 1,320,000 - 3,610,000                  	| High Price 	| 60 	|
    | 2 	| 301,000 - 777,000                      	| Low Price 	| 278 	|


### Visualize the resulting clusters

In [12]:
import numpy as np
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
!pip install folium
import pandas as pd
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

     |████████████████████████████████| 92kB 8.1MB/s eta 0:00:011


In [13]:
address = 'Melbourne, AU'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Melbourne is {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Melbourne is -37.8142176, 144.9631608.


In [14]:
# create map of Melbourne using latitude and longitude values
map_mel = folium.Map(location=[-37.8142176, 144.9631608], zoom_start=10)

k = 3
# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to map
markers_colors = []
for lat, lng, price, cluster in zip(df['Lattitude'], df['Longtitude'], df['Price'], df['Cluster']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_mel) 
    
map_mel

## Section 2. Find out top 5 venues in each neighborhood

In [24]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'LZOD4NAQQ053DIDXZPHCRDCDFDFTWXSXT1RV0BRPCRB4G0G1'
CLIENT_SECRET = 'QYISW0EZO1TQERTYEL3XOVKHVXSZQKCZGGKIITBBBAO5QLRW'
VERSION = '20180604'
LIMIT = 100
print ('Your credentials:')
print ('Client ID: ', CLIENT_ID)
print ('Client Secret', CLIENT_SECRET)

Your credentials:
Client ID:  LZOD4NAQQ053DIDXZPHCRDCDFDFTWXSXT1RV0BRPCRB4G0G1
Client Secret QYISW0EZO1TQERTYEL3XOVKHVXSZQKCZGGKIITBBBAO5QLRW


In [25]:
radius = 500 
latitude = df.loc[41,'Lattitude']
longitude = df.loc[41,'Longtitude']

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=LZOD4NAQQ053DIDXZPHCRDCDFDFTWXSXT1RV0BRPCRB4G0G1&client_secret=QYISW0EZO1TQERTYEL3XOVKHVXSZQKCZGGKIITBBBAO5QLRW&v=20180604&ll=-37.8078,144.9965&radius=500&limit=100'

In [28]:
import requests
import json
#Get the results of nearby venues of the first house 
def getNearbyVenues(names, latitudes, longitudes, radius=800):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        url    
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Suburb', 
                             'Suburb Latitude', 
                             'Suburb Longtitude',
                              'Venue', 
                              'Venue Latitude', 
                              'Venue Longitude', 
                              'Venue Category']
    
    return(nearby_venues)

In [29]:
location_venues = getNearbyVenues(names=df['Suburb'],
                                   latitudes=df['Lattitude'],
                                   longitudes=df['Longtitude']
                                  )
location_venues.head()

Abbotsford
Airport West
Altona
Altona
Altona
Altona
Altona
Altona
Altona
Armadale
Armadale
Armadale
Armadale
Ascot Vale
Ascot Vale
Ashwood
Avondale Heights
Avondale Heights
Balwyn
Balwyn North
Bentleigh
Bentleigh
Bentleigh
Bentleigh East
Bentleigh East
Bentleigh East
Bentleigh East
Bentleigh East
Bentleigh East
Bentleigh East
Bentleigh East
Bentleigh East
Bentleigh East
Bentleigh East
Bentleigh East
Box Hill
Braybrook
Brighton
Brighton
Brighton
Brighton
Brighton
Brighton
Brighton
Brighton
Brighton
Brighton East
Brighton East
Brunswick
Brunswick
Brunswick
Bulleen
Bulleen
Bulleen
Burwood
Burwood
Burwood
Burwood
Burwood
Burwood
Camberwell
Carnegie
Carnegie
Carnegie
Carnegie
Carnegie
Caulfield South
Caulfield South
Chadstone
Coburg
Collingwood
Collingwood
Collingwood
Collingwood
Doncaster
Doncaster
Doncaster
Doncaster
Doncaster
Doncaster
Doncaster
Doncaster
Eaglemont
Elsternwick
Elsternwick
Elwood
Essendon
Essendon
Essendon
Essendon
Essendon
Essendon
Essendon North
Essendon North
Fitzroy
F

,Suburb,Suburb Latitude,Suburb Longtitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Abbotsford,-37.8078,144.9965,Three Bags Full,-37.807318,144.996603,Café
1,Abbotsford,-37.8078,144.9965,Au79,-37.808806,144.996035,Café
2,Abbotsford,-37.8078,144.9965,Nhu Lan Bakery,-37.810375,144.996708,Bakery
3,Abbotsford,-37.8078,144.9965,Minh Phat Supermarket,-37.809652,144.996163,Grocery Store
4,Abbotsford,-37.8078,144.9965,Jinda Thai Restaurant,-37.809428,144.992345,Thai Restaurant


In [30]:
location_venues.groupby('Venue Category').count()

,Suburb,Suburb Latitude,Suburb Longtitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Accessories Store,3,3,3,3,3,3
Adult Boutique,3,3,3,3,3,3
Afghan Restaurant,5,5,5,5,5,5
African Restaurant,18,18,18,18,18,18
Airport,3,3,3,3,3,3
American Restaurant,16,16,16,16,16,16
Antique Shop,12,12,12,12,12,12
Aquarium,3,3,3,3,3,3
Arepa Restaurant,4,4,4,4,4,4


Analyze the area

In [31]:
melbourne_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")
melbourne_onehot['Suburb'] = location_venues['Suburb']
melbourne_onehot = melbourne_onehot.set_index('Suburb').reset_index()
melbourne_onehot.head()

,Suburb,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,...,Watch Shop,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yemeni Restaurant,Yoga Studio,Yunnan Restaurant,Zoo,Zoo Exhibit
0,Abbotsford,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Abbotsford,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Abbotsford,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Abbotsford,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Abbotsford,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
melbourne_grouped = melbourne_onehot.groupby('Suburb').mean().reset_index()
melbourne_grouped.head()

,Suburb,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,...,Watch Shop,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yemeni Restaurant,Yoga Studio,Yunnan Restaurant,Zoo,Zoo Exhibit
0,Abbotsford,0.0,0.012346,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aberfeldie,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Airport West,0.0,0.000000,0.0,0.0,0.065217,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Albion,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Altona,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
num_top_venues = 5

for hood in melbourne_grouped['Suburb']:
    print("----"+hood+"----")
    temp = melbourne_grouped[melbourne_grouped['Suburb'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))

----Abbotsford----
                   venue  freq
0  Vietnamese Restaurant  0.22
1                   Café  0.12
2        Thai Restaurant  0.07
3      Korean Restaurant  0.04
4                    Pub  0.04
----Aberfeldie----
                venue  freq
0  Athletics & Sports  0.22
1                Café  0.17
2         Golf Course  0.11
3                Park  0.11
4          Playground  0.11
----Airport West----
             venue  freq
0             Café  0.15
1    Grocery Store  0.09
2  Thai Restaurant  0.07
3          Airport  0.07
4       Food Truck  0.07
----Albion----
                   venue  freq
0                   Café  0.15
1            Music Store  0.12
2               Bus Stop  0.09
3                   Park  0.09
4  Vietnamese Restaurant  0.09
----Altona----
           venue  freq
0           Café  0.11
1    Pizza Place  0.10
2           Park  0.08
3          Beach  0.08
4  Train Station  0.08
----Altona Meadows----
                   venue  freq
0         Cricket Ground  0.3

In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
import numpy as np
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

columns = ['Suburb']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

melbourne_venues_sorted = pd.DataFrame(columns=columns)
melbourne_venues_sorted['Suburb'] = melbourne_grouped['Suburb']

for ind in np.arange(melbourne_grouped.shape[0]):
   melbourne_venues_sorted.iloc[ind, 1:] = return_most_common_venues(melbourne_grouped.iloc[ind, :], num_top_venues)

melbourne_venues_sorted.head()

,Suburb,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Abbotsford,Vietnamese Restaurant,Café,Thai Restaurant,Pub,Korean Restaurant
1,Aberfeldie,Athletics & Sports,Café,Park,Playground,Golf Course
2,Airport West,Café,Grocery Store,Airport,Thai Restaurant,Supermarket
3,Albion,Café,Music Store,Bus Stop,Park,Vietnamese Restaurant
4,Altona,Café,Pizza Place,Beach,Park,Train Station


In [44]:
melbourne_venues_sorted[melbourne_venues_sorted['1st Most Common Venue'] == 'Vietnamese Restaurant']

,Suburb,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Abbotsford,Vietnamese Restaurant,Café,Thai Restaurant,Pub,Korean Restaurant
20,Box Hill,Vietnamese Restaurant,Athletics & Sports,Baseball Field,Gourmet Shop,Supermarket
63,Footscray,Vietnamese Restaurant,Café,Asian Restaurant,Bakery,Bar


## Discussion and Conclusion

In our opinion, houses are clustered reasonably: Low-priced ones are still the majority, followed by the middle-priced. High price range is quite out of their capability, so we don’t discuss about it. 
As we can see from the map, the more expensive the property is, the closer it is to CBD (Central Business District) of Melbourne. 
There are 3 suburb areas that have Vietnamese restaurants within their proximity: Abbotsford, Box Hill, and Footscray. Of these three potential suburbs, Box Hill also has sports venues and one supermarket nearby, which is suitable for households. Moreover, Box Hill is in Low Price cluster  where housing price ranges from AUD 301,000 to AUD 777,000, which fits their financial capability.

In conclusion, the final recommendation for the Vietnamese client is **Box Hill, Melbourne, Australia**